# Table 4 Row 1 (Verify benchmark forces in the environment)

Load forces calculated from Du benchmark method

In [4]:
import pandas as pd

df = pd.read_csv('./Data/ADMM_Benchmark_Forces.csv', header=None)
forces = df.values.T
print(forces[0])

[-155.6882056     0.          120.4716266   123.1237685     0.
 -133.3444861     0.          283.5795504    85.86173852    0.
 -194.9797063     0.            0.            0.            0.
   98.67796165   56.27118196    3.89644975]


Apply forces in v22 environment

In [7]:
import numpy as np
import os
from os import path
import gym
from AssemblyGym.envs import FuselageActuators


env_id = "FuselageActuators-v22"

# Select files
file2 ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Benchmark/SolutionInputUndeformed.inp'
folder ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Test/'
files = os.listdir('./AssemblyGym/envs/FuselageActuators/AnsysFiles/Test/')


errors = []
maxDevs = []
meanForces = []
maxForces = []
# Loop over test samples
for i in range(19):
    print('*' * 30, f'Test sample {i+1}', '*' * 30)
    file1 = path.join(folder, files[i])
    env = gym.make(env_id, n_actuators=10, mode="File", file1=file1, file2=file2, record=False, seed=0, port=50056)
    cur_state = env.reset()
    done, r_tot = False, 0
    action = -forces[i]/1000
    next_state, reward, done, info = env.step(action)
    print("Reward:", reward)
    print("Final Error:", info["Error"])
    errors.append(info["Error"])
    maxDevs.append(info["maxDev"])
    meanForces.append(np.mean(forces))
    maxForces.append(np.max(forces))
    print('')
    env.close()   

print('*' * 30, 'Summary of Results', '*' * 30)
print("Mean RMSD = ", np.mean(errors))
print("Mean MD = ", np.mean(maxDevs))
print("Mean MF = ", np.mean(maxForces))

print("Max RMSD = ", np.max(errors))
print("Max MD = ", np.max(maxDevs))
print("Max MF = ", np.max(maxForces))

****************************** Test sample 1 ******************************
Exit Ansys and try to reconnect
No active Ansys process found. Wait and try to reconnect
Reconnect failed - remote exit again
Wait and try to reconnect again - attempt 1
Product:             Ansys Mechanical Enterprise Academic Teaching
MAPDL Version:       22.1
ansys.mapdl Version: 0.61.2

Running on 4 processors
Sucessfully reconnected to Ansys on attempt 2
Try running again
Simulation setup complete
Applied forces
Solve finished
Results ready
Exit Ansys and try to reconnect
Remote exit
Product:             Ansys Mechanical Enterprise Academic Teaching
MAPDL Version:       22.1
ansys.mapdl Version: 0.61.2

Running on 4 processors
Sucessfully reconnected to Ansys on attempt 1
Try running again
Simulation setup complete
Applied forces
Solve finished
Results ready
Reward: 0.9791273301370229
Final Error: 0.006240796302079029

****************************** Test sample 2 ******************************
Exit Ansys a

****************************** Summary of Results ******************************
**********************************************************

Mean RMSD =  0.011179177828747914

Mean MD =  0.024078412633001166

Mean MF =  289.53654900000004

**********************************************************

Max RMSD =  0.024337762854066206

Max MD =  0.04903018238336659

Max MF =  289.536549


# Table 4 Row 2 (Compare against forces from RL agent)

### Single shot adjustment

In [1]:
import os
from os import path
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import gym
import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions.normal import Normal

from AssemblyGym.envs import FuselageActuators

def make_env(env_id, seed, idx, n_actions, file1, file2, record):
    def thunk():
        env = gym.make(env_id, n_actuators=n_actions, mode="File", file1=file1, file2=file2, record=record, seed=seed, port=50056+idx)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        return env

    return thunk

def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer


class Agent(nn.Module):
    def __init__(self, envs):
        super().__init__()
        self.critic = nn.Sequential(
            layer_init(nn.Linear(np.array(envs.observation_space.shape).prod(), 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 1), std=1.0),
        )
        # layers for self.actor_mean
        self.fc1 = layer_init(nn.Linear(np.array(envs.observation_space.shape).prod(), 64))
        self.fc2 = layer_init(nn.Linear(64, 64))
        self.fc3 = layer_init(nn.Linear(64, np.prod(envs.action_space.shape)), std=0.01)
        
        self.actor_logstd = nn.Parameter(1*torch.ones(1, np.prod(envs.action_space.shape)), requires_grad=False)  # initial action_std = exp(actor_logstd)

    def get_value(self, obs):
        return self.critic(obs)

    def get_action_and_value(self, obs, action=None, scaleStd=1):
        # Start with standard MLP
        x = torch.tanh(self.fc1(obs))
        x = torch.tanh(self.fc2(x))
        action_mean = torch.tanh(self.fc3(x))
        # Build action distribution
        action_logstd = self.actor_logstd#.expand_as(action_mean)
        action_std = torch.exp(action_logstd)*scaleStd
        probs = Normal(action_mean, action_std)
        if action is None:
            action = probs.sample()
        if action == "deterministic":
            action = action_mean
        return action, probs.log_prob(action).sum(1), probs.entropy().sum(1), self.critic(obs)

# Make dummy environment to initialize agent against
env_name = "FuselageActuators-v22"
file2 ='./AssemblyGym/AssemblyGym/envs/FuselageActuators/AnsysFiles/Benchmark/SolutionInputUndeformed.inp'
file1 ='./AssemblyGym/AssemblyGym/envs/FuselageActuators/AnsysFiles/Benchmark/SolutionInputUndeformed.inp'
# envs = gym.vector.SyncVectorEnv(
#     [make_env(env_name, 0 + i, i, 10, file1, file2, False) for i in range(1)]
# )
envs = gym.make(env_name, n_actuators=10, mode="File", file1=file1, file2=file2, record=False, seed=0, port=50056)

# Create agent
device = torch.device("cpu")
agent = Agent(envs).to(device)
agent.load_state_dict(torch.load(
            "./Agents/agent_32767872steps.pt", map_location=device))
envs.close()

# Initialze variables
initErrors = []
finalErrors = []
maxDevs = []
maxForces = []
rewards = []

# Select files
file2 ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Benchmark/SolutionInputUndeformed.inp'
folder ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Test/'
files = os.listdir(folder)

# Loop over all files
for f in files[:-1]:
    file1 = path.join(folder, f)
    dp = file1[-8:-4] # Design point

    # Make the environment
    # env_name = "FuselageActuators-v22"
    # envs = gym.vector.SyncVectorEnv(
    #     [make_env(env_name, 0 + i, i, 10, file1, file2, False) for i in range(1)]
    # )
    print('*' * 30, f'File: {f}', '*' * 30)
    envs = gym.make(env_name, n_actuators=10, mode="File", file1=file1, file2=file2, record=False, seed=0, port=50056)

    
    # Perform test and track error
    obs = torch.Tensor(envs.reset()).to(device)
    #initErrors.append(envs.error_initial)
    episodeReward = 0
    # done=False
    # while not done:
    with torch.no_grad():
        action, logprob, _, value = agent.get_action_and_value(obs, action = "deterministic")
    obs, reward, done, info = envs.step(action.cpu().numpy())
    obs = torch.Tensor(obs).to(device)
    episodeReward += reward
    print("Reward:", episodeReward)
    print("Final Error:", info["Error"])
    initErrors.append(info["initError"])
    finalErrors.append(info["Error"])
    maxDevs.append(info["maxDev"])
    maxForces.append(np.max(np.abs(info["Forces"])))
    rewards.append(episodeReward)
    # n_actuators.append(np.count_nonzero(envs.forces))

    envs.close()

print("**********************************************************")
print("Initial error (mean) = %.3f" %np.mean(initErrors))
print("Initial error (median) = %.3f" %np.median(initErrors))
print("Initial error (stdev) = %.3f" %np.std(initErrors))
print("Initial error (max) = %.3f" %np.max(initErrors))
print("**********************************************************")
print("Final error (mean) = %.3f" %np.mean(finalErrors))
print("Final error (median) = %.3f" %np.median(finalErrors))
print("Final error (stdev) = %.3f" %np.std(finalErrors))
print("Final error (max) = %.3f" %np.max(finalErrors))
print("**********************************************************")
print("Max Deviation (mean) = %.3f" %np.mean(maxDevs))
print("Max Deviation (median) = %.3f" %np.median(maxDevs))
print("Max Deviation (stdev) = %.3f" %np.std(maxDevs))
print("Max Deviation (max) = %.3f" %np.max(maxDevs))
print("**********************************************************")
print("Max Force (mean) = %.3f" %np.mean(maxForces))
print("Max Force (median) = %.3f" %np.median(maxForces))
print("Max Force (stdev) = %.3f" %np.std(maxForces))
print("Max Force (max) = %.3f" %np.max(maxForces))
print("**********************************************************")
print("Episode Rewards (mean) = %.3f" %np.mean(rewards))
print("Episode Rewards (median) = %.3f" %np.median(rewards))
print("Episode Rewards (stdev) = %.3f" %np.std(rewards))

c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\scooby\knowledge.py:14: DeprecationWarning: The distutils.sysconfig module is deprecated, use sysconfig instead
  import distutils.sysconfig as sysconfig


PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
Product:             Ansys Mechanical Enterprise Academic Teaching
MAPDL Version:       22.1
ansys.mapdl Version: 0.64.0

Running on 4 processors


c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


****************************** File: SolutionInputDP41.inp ******************************
PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
Product:             Ansys Mechanical Enterprise Academic Teaching
MAPDL Version:       22.1
ansys.mapdl Version: 0.64.0

Running on 4 processors


c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\gym\utils\passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core envi

Reward: 0.9682654089509896
Final Error: 0.009488442052060824
****************************** File: SolutionInputDP42.inp ******************************
PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
Product:             Ansys Mechanical Enterprise Academic Teaching
MAPDL Version:       22.1
ansys.mapdl Version: 0.64.0

Running on 4 processors
Reward: 0.9377830113594768
Final Error: 0.009887509055903842
****************************** File: SolutionInputDP43.inp ******************************
PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
Product:             Ansys Mechanical Enterprise Academic Teaching
MAPDL Ver

c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\ansys\mapdl\core\launcher.py:1072: DeprecationWarning: invalid escape sequence '\d'
  and re.search("ansys\d\d\d", os.path.basename(os.path.normpath(exe_loc)))
c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\ansys\mapdl\core\launcher.py:1249: DeprecationWarning: invalid escape sequence '\-'
  """Start MAPDL locally.


MapdlConnectionError: ---- error analysis -----

---- error analysis -----


Expected Output
**********************************************************
Initial error (mean) = 0.467

Initial error (median) = 0.482

Initial error (stdev) = 0.251

Initial error (max) = 0.923
**********************************************************
Final error (mean) = 0.012

Final error (median) = 0.010

Final error (stdev) = 0.006

Final error (max) = 0.028
**********************************************************
Max Deviation (mean) = 0.037

Max Deviation (median) = 0.036

Max Deviation (stdev) = 0.013

Max Deviation (max) = 0.076
**********************************************************
Max Force (mean) = 42.936

Max Force (median) = 8.154

Max Force (stdev) = 56.856

Max Force (max) = 268.755
**********************************************************
Episode Rewards (mean) = 0.965

Episode Rewards (median) = 0.969

Episode Rewards (stdev) = 0.025

### Multiple refinements

In [2]:
import os
from os import path
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import gym
import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions.normal import Normal

from AssemblyGym.envs import FuselageActuators

def make_env(env_id, seed, idx, n_actions, file1, file2, record):
    env = gym.make(env_id, n_actuators=n_actions, mode="File", file1=file1, file2=file2, record=record, seed=seed, port=50056+idx)
    #env = gym.wrappers.RecordEpisodeStatistics(env)
    return env


def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer


class Agent(nn.Module):
    def __init__(self, envs):
        super().__init__()
        self.critic = nn.Sequential(
            layer_init(nn.Linear(np.array(envs.observation_space.shape).prod(), 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 1), std=1.0),
        )
        # layers for self.actor_mean
        self.fc1 = layer_init(nn.Linear(np.array(envs.observation_space.shape).prod(), 64))
        self.fc2 = layer_init(nn.Linear(64, 64))
        self.fc3 = layer_init(nn.Linear(64, np.prod(envs.action_space.shape)), std=0.01)
        
        self.actor_logstd = nn.Parameter(1*torch.ones(1, np.prod(envs.action_space.shape)), requires_grad=False)  # initial action_std = exp(actor_logstd)

    def get_value(self, obs):
        return self.critic(obs)

    def get_action_and_value(self, obs, action=None, scaleStd=1):
        # Start with standard MLP
        x = torch.tanh(self.fc1(obs))
        x = torch.tanh(self.fc2(x))
        action_mean = torch.tanh(self.fc3(x))
        # Build action distribution
        action_logstd = self.actor_logstd#.expand_as(action_mean)
        action_std = torch.exp(action_logstd)*scaleStd
        probs = Normal(action_mean, action_std)
        if action is None:
            action = probs.sample()
        if action == "deterministic":
            action = action_mean
        return action, probs.log_prob(action).sum(1), probs.entropy().sum(1), self.critic(obs)

# Make the environment
env_name = "FuselageActuators-v22"
file2 ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Benchmark/SolutionInputUndeformed.inp'
file1 ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Benchmark/SolutionInputUndeformed.inp'
# envs = gym.vector.SyncVectorEnv(
#     [make_env(env_name, 0 + i, i, 10, file1, file2, False) for i in range(1)]
# )

envs = make_env(env_name, 0, 0, 10, file1, file2, False)

# Create agent
device = torch.device("cpu")
agent = Agent(envs).to(device)
agent.load_state_dict(torch.load(
            "./Agents/agent_32767872steps.pt", map_location=device))

# Initialze variables
initErrors = []
finalErrors = []
maxDevs = []
maxForces = []
rewards = []
bestForces = []

# Select files
file2 ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Benchmark/SolutionInputUndeformed.inp'
folder ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Test/'
files = os.listdir(folder)

# Loop over all files
for f in files[:-1]:
    file1 = path.join(folder, f)
    dp = file1[-8:-4] # Design point

    # Make the environment
    # env_name = "FuselageActuators-v22"
    # envs = gym.vector.SyncVectorEnv(
    #     [make_env(env_name, 0 + i, i, 10, file1, file2, False) for i in range(1)]
    # )
    print('*' * 30, f'File: {f}', '*' * 30)
    envs = make_env(env_name, 0, 0, 10, file1, file2, False)

    # Perform test and track error
    obs = torch.Tensor(envs.reset()).to(device)
    # initErrors.append(envs.error_initial)
    episodeReward = 0
    minError=10
    
    for j in range(10):
        with torch.no_grad():
            action, logprob, _, value = agent.get_action_and_value(obs, action = "deterministic")
        obs, reward, done, info = envs.step(action.cpu().numpy())
        obs = torch.Tensor(obs).to(device)
        episodeReward += reward
        # print("Intermediate Reward:", reward)
        print("Intermediate Error:", info["Error"])
        if info["Error"]<minError:
            minError= info["Error"]
            maxDev = info["maxDev"]
            maxForce = np.max(np.abs(info["Forces"]))
            bestForce = info["Forces"]
    print('_'*30)
    print("Best Error:", minError)
    initErrors.append(info["initError"])
    finalErrors.append(minError)
    maxForces.append(maxForce)
    maxDevs.append(maxDev)
    rewards.append(episodeReward)
    bestForces.append(bestForce)

    envs.close()

print("**********************************************************")
print("Initial error (mean) = %.4f" %np.mean(initErrors))
print("Initial error (median) = %.4f" %np.median(initErrors))
print("Initial error (stdev) = %.4f" %np.std(initErrors))
print("Initial error (max) = %.4f" %np.max(initErrors))
print("**********************************************************")
print("Final error (mean) = %.4f" %np.mean(finalErrors))
print("Final error (median) = %.4f" %np.median(finalErrors))
print("Final error (stdev) = %.4f" %np.std(finalErrors))
print("Final error (max) = %.4f" %np.max(finalErrors))
print("**********************************************************")
print("Max Deviation (mean) = %.4f" %np.mean(maxDevs))
print("Max Deviation (median) = %.4f" %np.median(maxDevs))
print("Max Deviation (stdev) = %.4f" %np.std(maxDevs))
print("Max Deviation (max) = %.4f" %np.max(maxDevs))
print("**********************************************************")
print("Max Force (mean) = %.4f" %np.mean(maxForces))
print("Max Force (median) = %.4f" %np.median(maxForces))
print("Max Force (stdev) = %.4f" %np.std(maxForces))
print("Max Force (max) = %.4f" %np.max(maxForces))
print("**********************************************************")
print("Best Forces:", bestForces)


****************************** File: SolutionInputDP41.inp ******************************


c:\Users\TL\Python\venv\pyANSYS\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\TL\Python\venv\pyANSYS\lib\site-packages\gym\utils\passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
c:\Users\TL\Python\venv\pyANSYS\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(


Exit Ansys and try to reconnect
No active Ansys process found. Wait and try to reconnect
Product:             Ansys Mechanical Enterprise Academic Research
MAPDL Version:       22.1
ansys.mapdl Version: 0.61.2

Running on 4 processors
Sucessfully reconnected to Ansys on attempt 1
Try running again
Simulation setup complete
Applied forces
Solve finished
Results ready


c:\Users\TL\Python\venv\pyANSYS\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


Intermediate Error: 0.009488442052060824
Intermediate Error: 0.009219268499272492
Intermediate Error: 0.008870858205252264
Intermediate Error: 0.008603489609872469
Intermediate Error: 0.1496927783657662
Intermediate Error: 0.008271135366351488
Intermediate Error: 0.5035429930857178
Intermediate Error: 0.06381492841905549
Intermediate Error: 0.008625910182616174
Intermediate Error: 0.4496713490799514
______________________________
Best Error: 0.008271135366351488
****************************** File: SolutionInputDP42.inp ******************************
Exit Ansys and try to reconnect
No active Ansys process found. Wait and try to reconnect
Product:             Ansys Mechanical Enterprise Academic Research
MAPDL Version:       22.1
ansys.mapdl Version: 0.61.2

Running on 4 processors
Sucessfully reconnected to Ansys on attempt 1
Try running again
Simulation setup complete
Applied forces
Solve finished
Results ready
Intermediate Error: 0.009887509055903842
Intermediate Error: 0.00884927917

**********************************************************
Initial error (mean) = 0.4673
Initial error (median) = 0.4824
Initial error (stdev) = 0.2508
Initial error (max) = 0.9233
**********************************************************
Final error (mean) = 0.0099
Final error (median) = 0.0094
Final error (stdev) = 0.0028
Final error (max) = 0.0162
**********************************************************
Max Deviation (mean) = 0.0331
Max Deviation (median) = 0.0338
Max Deviation (stdev) = 0.0077
Max Deviation (max) = 0.0465
**********************************************************
Max Force (mean) = 274.0799
Max Force (median) = 255.0113
Max Force (stdev) = 96.0210
Max Force (max) = 476.3623
**********************************************************

### Multiple refinements (test)

In [ ]:
import os
from os import path
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import gym
import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions.normal import Normal

from AssemblyGym.envs import FuselageActuators

def make_env(env_id, seed, idx, n_actions, file1, file2, record):
    env = gym.make(env_id, n_actuators=n_actions, mode="Test", record=record, seed=seed)
    #env = gym.wrappers.RecordEpisodeStatistics(env)
    return env


def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer


class Agent(nn.Module):
    def __init__(self, envs):
        super().__init__()
        self.critic = nn.Sequential(
            layer_init(nn.Linear(np.array(envs.observation_space.shape).prod(), 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 1), std=1.0),
        )
        # layers for self.actor_mean
        self.fc1 = layer_init(nn.Linear(np.array(envs.observation_space.shape).prod(), 64))
        self.fc2 = layer_init(nn.Linear(64, 64))
        self.fc3 = layer_init(nn.Linear(64, np.prod(envs.action_space.shape)), std=0.01)
        
        self.actor_logstd = nn.Parameter(-5*torch.ones(1, np.prod(envs.action_space.shape)), requires_grad=False)  # initial action_std = exp(actor_logstd)

    def get_value(self, obs):
        return self.critic(obs)

    def get_action_and_value(self, obs, action=None, scaleStd=1):
        # Start with standard MLP
        x = torch.tanh(self.fc1(obs))
        x = torch.tanh(self.fc2(x))
        action_mean = torch.tanh(self.fc3(x))
        # Build action distribution
        action_logstd = self.actor_logstd#.expand_as(action_mean)
        action_std = torch.exp(action_logstd)*scaleStd
        probs = Normal(action_mean, action_std)
        if action is None:
            action = probs.sample()
        if action == "deterministic":
            action = action_mean
        return action, probs.log_prob(action).sum(1), probs.entropy().sum(1), self.critic(obs)

# Make the environment
env_name = "FuselageActuators-v22"
file2 ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Benchmark/SolutionInputUndeformed.inp'
file1 ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Benchmark/SolutionInputUndeformed.inp'
# envs = gym.vector.SyncVectorEnv(
#     [make_env(env_name, 0 + i, i, 10, file1, file2, False) for i in range(1)]
# )

envs = make_env(env_name, 0, 0, 10, file1, file2, False)

# Create agent
device = torch.device("cpu")
agent = Agent(envs).to(device)
agent.load_state_dict(torch.load(
            "./Agents/agent_32767872steps.pt", map_location=device))

# Initialze variables
initErrors = []
finalErrors = []
maxDevs = []
maxForces = []
rewards = []
bestForces = []

# Select files
file2 ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Benchmark/SolutionInputUndeformed.inp'
folder ='./AssemblyGym/envs/FuselageActuators/AnsysFiles/Test/'
files = os.listdir(folder)

# Loop over all files
for f in files[:-1]:
    file1 = path.join(folder, f)
    dp = file1[-8:-4] # Design point

    # Make the environment
    # env_name = "FuselageActuators-v22"
    # envs = gym.vector.SyncVectorEnv(
    #     [make_env(env_name, 0 + i, i, 10, file1, file2, False) for i in range(1)]
    # )

    envs = make_env(env_name, 0, 0, 10, file1, file2, False)

    print('*' * 30, f'File: {f}', '*' * 30)
    # Perform test and track error
    obs = torch.Tensor(envs.reset()).to(device)
    # initErrors.append(envs.error_initial)
    episodeReward = 0
    minError=10
    
    for j in range(10):
        with torch.no_grad():
            action, logprob, _, value = agent.get_action_and_value(obs, action = "deterministic")
        obs, reward, done, info = envs.step(action.cpu().numpy())
        obs = torch.Tensor(obs).to(device)
        episodeReward += reward
        # print("Intermediate Reward:", reward)
        print("Intermediate Error:", info["Error"])
        if info["Error"]<minError:
            minError= info["Error"]
            maxDev = info["maxDev"]
            maxForce = np.max(np.abs(info["Forces"]))
            bestForce = info["Forces"]
    print('_'*30)
    print("Best Error:", minError)
    initErrors.append(info["initError"])
    finalErrors.append(minError)
    maxForces.append(maxForce)
    maxDevs.append(maxDev)
    rewards.append(episodeReward)
    bestForces.append(bestForce)

    envs.close()

print("**********************************************************")
print("Initial error (mean) = %.4f" %np.mean(initErrors))
print("Initial error (median) = %.4f" %np.median(initErrors))
print("Initial error (stdev) = %.4f" %np.std(initErrors))
print("Initial error (max) = %.4f" %np.max(initErrors))
print("**********************************************************")
print("Final error (mean) = %.4f" %np.mean(finalErrors))
print("Final error (median) = %.4f" %np.median(finalErrors))
print("Final error (stdev) = %.4f" %np.std(finalErrors))
print("Final error (max) = %.4f" %np.max(finalErrors))
print("**********************************************************")
print("Max Deviation (mean) = %.4f" %np.mean(maxDevs))
print("Max Deviation (median) = %.4f" %np.median(maxDevs))
print("Max Deviation (stdev) = %.4f" %np.std(maxDevs))
print("Max Deviation (max) = %.4f" %np.max(maxDevs))
print("**********************************************************")
print("Max Force (mean) = %.4f" %np.mean(maxForces))
print("Max Force (median) = %.4f" %np.median(maxForces))
print("Max Force (stdev) = %.4f" %np.std(maxForces))
print("Max Force (max) = %.4f" %np.max(maxForces))
print("**********************************************************")
print("Best Forces:", bestForces)


c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\scooby\knowledge.py:14: DeprecationWarning: The distutils.sysconfig module is deprecated, use sysconfig instead
  import distutils.sysconfig as sysconfig


PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...


c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\ansys\mapdl\core\launcher.py:1072: DeprecationWarning: invalid escape sequence '\d'
  and re.search("ansys\d\d\d", os.path.basename(os.path.normpath(exe_loc)))
c:\Users\TL\Documents\PythonVENV\venv3_10-pyAnsys\lib\site-packages\ansys\mapdl\core\launcher.py:1249: DeprecationWarning: invalid escape sequence '\-'
  """Start MAPDL locally.


KeyboardInterrupt: 

Expected output
**********************************************************
Initial error (mean) = 0.467
Initial error (median) = 0.482
Initial error (stdev) = 0.251
Initial error (max) = 0.923
**********************************************************
Final error (mean) = 0.010
Final error (median) = 0.010
Final error (stdev) = 0.003
Final error (max) = 0.015
**********************************************************
Max Deviation (mean) = 0.034
Max Deviation (median) = 0.035
Max Deviation (stdev) = 0.009
Max Deviation (max) = 0.056
**********************************************************
Max Force (mean) = 214.811
Max Force (median) = 207.556
Max Force (stdev) = 89.130
Max Force (max) = 432.262
**********************************************************